# Loan Payment Prediction
Data set description
Question: Will the borrower pay the loan? - Prediction

Columns to take a look at:
- Columns that leak info from the future -to be avoided in our model
- Columns that don't impact the borrower's ability to pay loan
- Columns that need data cleaning
- Missing data
- Combination of columns to get features
- Redundant columns

Which is our target column to predict the loan payment? `loan_status`

In [1]:
import pandas as pd
pd.options.display.max_columns = 52

In [2]:
loans_2007 = pd.read_csv('loans_2007.csv')

/home/asis/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loans_2007.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [4]:
loans_2007.shape

(42538, 52)

In [5]:
relevant_columns = ['loan_amnt','term','int_rate','installment',
                   'emp_length','home_ownership','annual_inc',
                   'pymnt_plan','purpose','addr_state','delinq_2yrs',
                   'earliest_cr_line','inq_last_6mths',
                    'open_acc','pub_rec','total_acc','application_type',
                    'tax_liens','loan_status' #target col
                   ]
# grade and sub_grade not included since they include redundant info from int_rate
# earliest_cr_line : create new feature: years since first credit line
#inq_last_ [6 and 12] months: figure sth out
relevant_columns
drop_columns = loans_2007.columns.difference(relevant_columns)
drop_columns

Index(['acc_now_delinq', 'chargeoff_within_12_mths', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'delinq_amnt', 'dti', 'emp_title',
       'funded_amnt', 'funded_amnt_inv', 'grade', 'id', 'initial_list_status',
       'issue_d', 'last_credit_pull_d', 'last_pymnt_amnt', 'last_pymnt_d',
       'member_id', 'out_prncp', 'out_prncp_inv', 'policy_code',
       'pub_rec_bankruptcies', 'recoveries', 'revol_bal', 'revol_util',
       'sub_grade', 'title', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int',
       'total_rec_late_fee', 'total_rec_prncp', 'verification_status',
       'zip_code'],
      dtype='object')

In [6]:
loans_2007=loans_2007.drop(drop_columns,axis=1) #drop columns

In [7]:
loans_2007['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

In [8]:
# keep loans fully paid or charged off only
loans_2007 = loans_2007[
    (loans_2007['loan_status'] == 'Fully Paid') |
    (loans_2007['loan_status'] == 'Charged Off') 
     ]

# transform fully paid and charged off to numeric values
mapping_dict = {
    'loan_status': {
        'Fully Paid': 1,
        'Charged Off': 0
    }
}
loans_2007 = loans_2007.replace(mapping_dict)

In [11]:
loans_2007['loan_status'].value_counts() # confirm changes

1    33136
0     5634
Name: loan_status, dtype: int64

There are way more paid off than charged off loans, which means that our model will tend to predict more 1 (paid off) loans than charged off - biased model.

In [13]:
loans_2007['tax_liens'].unique()

array([ 0., nan])

In [17]:
# drop columns that contain only 1 unique value besides null

drop_columns = []

for col in loans_2007.columns:
    non_null_vals = loans_2007[col].dropna()
    unique_vals = non_null_vals.unique()
    num_unique_vals = len(unique_vals)
    if num_unique_vals == 1:
        drop_columns.append(col)

loans_2007 = loans_2007.drop(drop_columns,axis=1)
drop_columns # to see which columns were dropped
loans_2007.columns

# DQ COLUMNS
# ['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
#        'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
#        'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
#        'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
#        'revol_bal', 'revol_util', 'total_acc', 'last_credit_pull_d',
#        'pub_rec_bankruptcies']

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'loan_status', 'purpose', 'addr_state',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'total_acc'],
      dtype='object')